In [1]:
import pandas as pd
import numpy as np

# Grouping and prorcessing demand data

In [3]:
path = "../data/"
demand_path = f"{path}raw/yellow_tripdata_2025-01.parquet"

In [4]:
try:
    demand_df = pd.read_parquet(demand_path)
    print(f"Data loaded successfully. Shape: {demand_df.shape}")
except FileNotFoundError:
    print(f"Error: File not found at {demand_path}")
    demand_df = None

Data loaded successfully. Shape: (3475226, 19)


### Data Cleaning

In [6]:
try:
    demand_df['pickup_datetime'] = pd.to_datetime(demand_df['tpep_pickup_datetime'])
    demand_df['dropoff_datetime'] = pd.to_datetime(demand_df['tpep_dropoff_datetime'])
    print("Datetime columns converted.")
except Exception as e:
    print(f"Error converting datetime columns: {e}")

# Remove invalid datetimes (keeping it simple, assuming 'pickup' should not be after 'dropoff')
initial_rows = len(demand_df)
demand_df = demand_df[demand_df['pickup_datetime'] <= demand_df['dropoff_datetime']]
rows_removed = initial_rows - len(demand_df)
if rows_removed > 0:
    print(f"Removed {rows_removed} rows with invalid datetime order.")
else:
    print("No rows removed due to invalid datetime order.")

# Handle NaN values in critical columns
cols_to_check = ['total_amount', 'PULocationID', 'DOLocationID']
initial_rows = len(demand_df)
demand_df.dropna(subset=cols_to_check, inplace=True)
rows_removed = initial_rows - len(demand_df)
if rows_removed > 0:
    print(f"Removed {rows_removed} rows with NaN in: {cols_to_check}.")
else:
    print(f"No rows removed due to NaN in: {cols_to_check}.")

# Handle NaN in passenger_count
initial_nan_count = demand_df['passenger_count'].isnull().sum()
demand_df['passenger_count'].fillna(1, inplace=True)
filled_nan_count = initial_nan_count - demand_df['passenger_count'].isnull().sum()
if filled_nan_count > 0:
    print(f"Filled {filled_nan_count} NaN values in 'passenger_count' with 1.")
else:
    print("No NaN values found in 'passenger_count'.")

print(f"Cleaned DataFrame shape: {demand_df.shape}")

Datetime columns converted.
Removed 124 rows with invalid datetime order.
No rows removed due to NaN in: ['total_amount', 'PULocationID', 'DOLocationID'].
Filled 540031 NaN values in 'passenger_count' with 1.
Cleaned DataFrame shape: (3475102, 21)


C:\Users\leona\AppData\Local\Temp\ipykernel_14628\1597511884.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  demand_df['passenger_count'].fillna(1, inplace=True)


In [7]:
demand_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,pickup_datetime,dropoff_datetime
0,1,2025-01-01 00:18:38,2025-01-01 00:26:59,1.0,1.60,1.0,N,229,237,1,...,3.5,0.5,3.00,0.0,1.0,18.00,2.5,0.0,2025-01-01 00:18:38,2025-01-01 00:26:59
1,1,2025-01-01 00:32:40,2025-01-01 00:35:13,1.0,0.50,1.0,N,236,237,1,...,3.5,0.5,2.02,0.0,1.0,12.12,2.5,0.0,2025-01-01 00:32:40,2025-01-01 00:35:13
2,1,2025-01-01 00:44:04,2025-01-01 00:46:01,1.0,0.60,1.0,N,141,141,1,...,3.5,0.5,2.00,0.0,1.0,12.10,2.5,0.0,2025-01-01 00:44:04,2025-01-01 00:46:01
3,2,2025-01-01 00:14:27,2025-01-01 00:20:01,3.0,0.52,1.0,N,244,244,2,...,1.0,0.5,0.00,0.0,1.0,9.70,0.0,0.0,2025-01-01 00:14:27,2025-01-01 00:20:01
4,2,2025-01-01 00:21:34,2025-01-01 00:25:06,3.0,0.66,1.0,N,244,116,2,...,1.0,0.5,0.00,0.0,1.0,8.30,0.0,0.0,2025-01-01 00:21:34,2025-01-01 00:25:06


### Time Interval Conversion and Day Type

In [9]:
# Floor datetime to 15-minute intervals and extract time without seconds
demand_df['pickup_time'] = demand_df['pickup_datetime'].dt.floor('15min').dt.time.apply(lambda t: t.strftime('%H:%M'))
demand_df['dropoff_time'] = demand_df['dropoff_datetime'].dt.floor('15min').dt.time.apply(lambda t: t.strftime('%H:%M'))


# Add day type columns
demand_df['pickup_day_type'] = demand_df['pickup_datetime'].dt.day_name().apply(
    lambda day: 'weekday' if day not in ['Saturday', 'Sunday'] else day.lower()
)
demand_df['dropoff_day_type'] = demand_df['dropoff_datetime'].dt.day_name().apply(
    lambda day: 'weekday' if day not in ['Saturday', 'Sunday'] else day.lower()
)

### Grouping and Aggregation

In [11]:
aggregated_df = demand_df.groupby(['PULocationID', 'DOLocationID', 'pickup_time', 'dropoff_time']).agg(
    num_trips=('tpep_pickup_datetime', 'count'),  # Count the number of trips (rows)
    sum_passenger_count=('passenger_count', 'sum'),
    mean_passenger_count=('passenger_count', 'mean'),
    sum_total_amount=('total_amount', 'sum'),
    mean_total_amount=('total_amount', 'mean')
).reset_index()

print(f"Aggregated DataFrame shape: {aggregated_df.shape}")
print(aggregated_df.head())

Aggregated DataFrame shape: (702982, 9)
   PULocationID  DOLocationID pickup_time dropoff_time  num_trips  \
0             1             1       01:00        01:00          1   
1             1             1       02:30        02:30          1   
2             1             1       03:15        03:15          1   
3             1             1       03:30        03:30          1   
4             1             1       03:45        03:45          1   

   sum_passenger_count  mean_passenger_count  sum_total_amount  \
0                  1.0                   1.0              73.2   
1                  1.0                   1.0              99.0   
2                  2.0                   2.0             133.2   
3                  1.0                   1.0             111.0   
4                  2.0                   2.0             139.2   

   mean_total_amount  
0               73.2  
1               99.0  
2              133.2  
3              111.0  
4              139.2  


In [12]:
aggregated_df

,PULocationID,DOLocationID,pickup_time,dropoff_time,num_trips,sum_passenger_count,mean_passenger_count,sum_total_amount,mean_total_amount
0,1,1,01:00,01:00,1,1.0,1.000000,73.20,73.200000
1,1,1,02:30,02:30,1,1.0,1.000000,99.00,99.000000
2,1,1,03:15,03:15,1,2.0,2.000000,133.20,133.200000
3,1,1,03:30,03:30,1,1.0,1.000000,111.00,111.000000
4,1,1,03:45,03:45,1,2.0,2.000000,139.20,139.200000
...,...,...,...,...,...,...,...,...,...
702977,265,265,23:15,23:15,13,19.0,1.461538,1484.01,114.154615
702978,265,265,23:30,23:30,17,30.0,1.764706,1860.14,109.420000
702979,265,265,23:30,23:45,5,5.0,1.000000,560.40,112.080000
702980,265,265,23:45,00:00,1,1.0,1.000000,138.07,138.070000


In [21]:
aggregated_df.to_csv(f"{path}/processed/grouped_zone_demand.csv")